In [1]:
import h5py
import sys
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
sys.path.append('../')
import utils
import numpy as np
import haiku as hk
import jax
import jax.numpy as jnp
from nucleotide_transformer.pretrained import get_pretrained_model
from tqdm import tqdm
import json
import tensorflow as tf
model_name = '2B5_1000G'
embed_layer = 32
include_seq = True

In [2]:
file = h5py.File('../../data/rna_stable/insert_dataset.h5','r')
L = file['X_test'].shape[1]

In [6]:
len(file['X_test']) + len(file['X_train']) + len(file['X_valid'])

11417

In [3]:
parameters, forward_fn, tokenizer, config = get_pretrained_model(
    model_name=model_name,
    mixed_precision=False,
    embeddings_layers_to_save=(embed_layer,),
    attention_maps_to_save=(),
    max_positions=34,
)
forward_fn = hk.transform(forward_fn)

In [4]:
transformer_output = h5py.File('../../data/rna_stable/2B5_1000G_embed.h5','w')
random_key = jax.random.PRNGKey(0)
b_size = 1024

for dataset in ['test','train','valid']:
    key = 'X_'+dataset
    print(key)
    onehot = file[key]
    string_seq = utils.onehot_to_seq(onehot)
    token_seq = tokenizer.batch_tokenize(string_seq)
    token_id = [b[1] for b in token_seq]
    seq_batch = jnp.asarray(token_id,dtype=jnp.int32)
    out_embed = []
    for i in tqdm(range(0,len(seq_batch),b_size)):
        input_seq = seq_batch[i:i+b_size]
        outs = forward_fn.apply(parameters, random_key, input_seq)
        out_embed.extend(np.array(outs['embeddings_'+str(embed_layer)]))

    transformer_output.create_dataset(name = key, data = out_embed)
    transformer_output.create_dataset(name='Y_'+dataset,data = file['Y_'+dataset][:])

    

X_test


100%|██████████| 2/2 [00:11<00:00,  5.86s/it]


X_train


100%|██████████| 9/9 [00:29<00:00,  3.33s/it]


X_valid


100%|██████████| 2/2 [00:07<00:00,  3.53s/it]


In [5]:
transformer_output['X_train'].shape

(9131, 34, 2560)

In [6]:
file['X_train'].shape

(9131, 173, 4)

In [8]:
transformer_output.close()